<a href="https://colab.research.google.com/github/Mahdi-Saadati/Basic-Machine-Learning/blob/main/16.a.%20Reinforcement%20Learning%20(RL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" align="right">
یادگیری تقویتی (Reinforcement Learning)
هدف: پیش‌بینی تصمیم دارویی (دارو بده / نده) بر اساس ویژگی‌های بیمار و یادگیری بهترین استراتژی.

مراحل کار:

بارگذاری دیتاست بیماران دیابتی.

ساده‌سازی دیتاست برای حالت‌های گسسته (سه وضعیت بیمار).

تعریف محیط RL ساده: هر بیمار = حالت، دارو دادن یا ندادن = اکشن، پاداش = بهبود یا عدم بهبود بیمار.

اجرای الگوریتم Q-Learning.

In [2]:
# 1. وارد کردن کتابخانه‌های مورد نیاز
import numpy as np
import random
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import KBinsDiscretizer

# 2. بارگذاری دیتاست دیابت از sklearn
data = load_diabetes()
X = data.data      # ویژگی‌های بیماران (مثل سن، فشار خون، BMI و ...)
y = data.target    # هدف: شدت بیماری دیابت (عدد پیوسته)

# 3. تبدیل هدف عددی به سه دسته گسسته: بدحال(0)، متوسط(1)، خوب(2)
y = y.reshape(-1, 1)  # تبدیل به ماتریس ستونی برای KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
states = kbd.fit_transform(y).astype(int).flatten()  # برچسب گسسته برای هر بیمار

# 4. تعداد بیماران
n_patients = len(states)

# 5. تعریف اکشن‌ها: 0 = دارو نده، 1 = دارو بده
actions = [0, 1]

# 6. ایجاد جدول Q با ابعاد [تعداد بیماران × تعداد اکشن‌ها] و مقدار اولیه صفر
Q = np.zeros((n_patients, len(actions)))

# 7. پارامترهای الگوریتم Q-Learning
alpha = 0.1   # نرخ یادگیری (چقدر Q به روز شود)
gamma = 0.9   # ضریب تخفیف (چقدر به آینده اهمیت بدهیم)
epsilon = 0.2 # احتمال انتخاب تصادفی (اکتشاف)
episodes = 100 # تعداد بار تمرین روی دیتاست

# 8. تابع پاداش ساده
def reward(state, action):
    # اگر دارو بدهیم و بیمار خوب نشده باشد → پاداش 5
    if action == 1 and state < 2:
        return 5
    # اگر دارو ندهیم و بیمار خوب است → پاداش 2
    elif action == 0 and state == 2:
        return 2
    # بقیه حالت‌ها → پاداش منفی (-1)
    else:
        return -1

# 9. حلقه آموزش Q-Learning
for episode in range(episodes):
    for i, state in enumerate(states):  # بررسی هر بیمار
        # انتخاب اکشن با روش ε-greedy
        if random.uniform(0,1) < epsilon:
            action = random.choice(actions)  # اکتشاف: انتخاب تصادفی
        else:
            action = np.argmax(Q[i])         # بهره‌برداری: بهترین اکشن فعلی

        # دریافت پاداش
        r = reward(state, action)

        # حالت بعدی (برای سادگی همان حالت فعلی است)
        next_state = state

        # آپدیت جدول Q
        Q[i, action] += alpha * (r + gamma * np.max(Q[i]) - Q[i, action])

# 10. نمایش سیاست یادگرفته شده برای 10 بیمار اول
for i, state in enumerate(states[:10]):
    best_action = np.argmax(Q[i])
    print(f"بیمار {i} وضعیت {state} → بهترین تصمیم: {'دارو بده' if best_action==1 else 'دارو نده'}")


بیمار 0 وضعیت 1 → بهترین تصمیم: دارو بده
بیمار 1 وضعیت 0 → بهترین تصمیم: دارو بده
بیمار 2 وضعیت 1 → بهترین تصمیم: دارو بده
بیمار 3 وضعیت 1 → بهترین تصمیم: دارو بده
بیمار 4 وضعیت 1 → بهترین تصمیم: دارو بده
بیمار 5 وضعیت 0 → بهترین تصمیم: دارو بده
بیمار 6 وضعیت 1 → بهترین تصمیم: دارو بده
بیمار 7 وضعیت 0 → بهترین تصمیم: دارو بده
بیمار 8 وضعیت 0 → بهترین تصمیم: دارو بده
بیمار 9 وضعیت 2 → بهترین تصمیم: دارو نده


<div dir="rtl" align="right">

توضیح ساده:

هر بیمار = یک حالت (بسته به شدت بیماری).

اکشن = دارو دادن یا ندادن.

پاداش = بهبود یا عدم بهبود بیمار.

الگوریتم Q-Learning یاد می‌گیرد که برای هر بیمار بهترین تصمیم چیست.